# Mask R-CNN inference

In [1]:
import tensorflow as tf
from keras import backend as K
import os
import sys
import random
import math
import copy
import json
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import matplotlib.image as img
from matplotlib.patches import Rectangle
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
import samples.coco.coco as coco
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean
from mrcnn.utils import Dataset
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
from mrcnn.model import MaskRCNN
from mrcnn.config import Config
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from mrcnn import visualize
from mrcnn.model import log


%matplotlib inline 

# cesta pro ulozeni vysledku
MODEL_DIR = 'training_summaries/'
# cesta k natrenovanemu modelu
MODEL_PATH = "training_summaries/mask_rcnn_coralclef2020_epoch-083.h5"
# slozka s obrazky pro detekci
IMAGE_DIR = "images/"
SAVE_DIR = 'images_out/'

Using TensorFlow backend.


In [2]:
class InferenceConfig(coco.CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 13 + 1
    DETECTION_MIN_CONFIDENCE = 0.5

config = InferenceConfig()
config.display()

# CoralClef 2020 jmena trid
class_names = ['BG', 'hard coral branching', 'hard coral submassive', 'hard coral boulder',
           'hard coral encrusting', 'hard coral table', 'hard coral foliose', 'hard coral mushroom',
           'soft coral', 'soft coral gorgonian', 'sponge', 'sponge barrel', 'fire coral millepora',
           'algae macro or leaves']


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                26
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [3]:
K.clear_session()

# vytvoreni modelu v modu inference
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# nacteni natrenovaneho modelu
model.load_weights(MODEL_PATH, by_name=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.



## Detekce na jednotlivých obrázcích

In [ ]:
c = [(1.0, 0.9230769230769231, 0.0),
 (1.0, 0.46153846153846156, 0.0),
 (0.0, 0.3076923076923075, 1.0),
 (1.0, 0.0, 0.9230769230769234),
 (0.1538461538461533, 0.0, 1.0),
 (0.6153846153846154, 1.0, 0.0),
 (0.0, 1.0, 0.30769230769230793),
 (0.0, 0.7692307692307692, 1.0),
 (0.0, 1.0, 0.7692307692307692),
 (1.0, 0.0, 0.0),
 (0.15384615384615374, 1.0, 0.0),
 (0.6153846153846159, 0.0, 1.0),
 (1.0, 0.0, 0.4615384615384617),
 (1.0, 0.9230769230769231, 0.1)]



# nacteni obrazku
file_names = os.listdir(IMAGE_DIR)
file_name = file_names[random.randint(0, len(file_names)-1)]
image = skimage.io.imread(os.path.join(IMAGE_DIR, file_name))

# detekce
results = model.detect([image], verbose=1)
r = results[0]

colors = []
for i in r['class_ids']:
    colors.append(c[i-1])

# zobrazeni
img = visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'], colors=colors, save_fig = SAVE_DIR + file_name)

## Detekce na validační a trénovací množině

In [15]:
# cesta k datasetu a anotacím
DATASET_DIR = "../CoralClef2020/training_set_2020/"
ANNOTATION_TRAIN = '../CoralClef2020/annotations/train_annot.json'
ANNOTATION_VAL =  '../CoralClef2020/annotations/validation_annot.json'


In [16]:
# trida pro nacteni datasetu
class CoralDataset(Dataset):
    
    # nacteni obrazku
    def load_dataset(self, is_train=True):

        # pridani trid do datasetu
        for i in range(1,len(class_names)):
            self.add_class("dataset", i, class_names[i])
        
        # nastaveni cesty k souboru s anotacemi
        if(is_train):
            annotation_path = ANNOTATION_TRAIN
        else:
            annotation_path = ANNOTATION_VAL
            
        with open(annotation_path) as json_file:
            data = json.load(json_file)
        
        # nacteni anotaci a vlozeni obrazku do datasetu
        for annot in data['images']:
            image_id = annot['id']
            img_path = DATASET_DIR + annot['file_name']
            ann_path = annotation_path

            self.add_image('dataset', 
                           image_id=image_id, 
                           width=annot['width'], height=annot['height'],
                           path=img_path, 
                           annotation=ann_path)
            


    # nacteni masek
    def load_mask(self, image_id):
        # nacteni informaci o obrazku
        info = self.image_info[image_id]

        # nacteni anotaci
        path = info['annotation']
        with open(path) as json_file:
            data = json.load(json_file)
        
        segments = []
        class_ids = []
        for s in data['annotations']:
            if(s['image_id'] == info['id']):
                segments.append(s['segmentation'])
                class_ids.append(s['category_id'] + 1)
            
        # vytvoreni 2d pole pro kazdou masku
        masks = zeros([info['height'], info['width'], len(segments)], dtype='uint8')
        for i in range(len(segments)):
            r = []
            c = []
            for s0,s1 in zip(segments[i][0::2], segments[i][1::2]):
                r.append(int(s1))
                c.append(int(s0))
            r = np.array(r)
            c = np.array(c)
            rr, cc = skimage.draw.polygon(r, c)
            masks[rr, cc, i] = 1
        return masks, asarray(class_ids, dtype='int32')
 
    # nacteni informaci o obrazku
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']    
    
    

In [17]:
# nacteni trenovaci mnoziny
train_set = CoralDataset()
train_set.load_dataset(is_train=True)
train_set.prepare()
print('Trenovaci: %d' % len(train_set.image_ids))

# nacteni validacni mnoziny
val_set = CoralDataset()
val_set.load_dataset(is_train=False)
val_set.prepare()
print('Validacni: %d' % len(val_set.image_ids))

Trenovaci: 371
Validacni: 69


In [ ]:
# provede detekci na obrazcich z validacni mnoziny
# image_ids = list() - specifikuje na jkaych obrazcich ma byt provedena detekce

dataset = val_set



#image_ids = np.random.choice(dataset.image_ids, 20)
image_ids = dataset.image_ids 

progbar = tf.keras.utils.Progbar(len(image_ids))
k = 0

for image_id in image_ids:
    progbar.update(k)
    # nacteni ground truth dat
    image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(dataset, config, 
                                                                              image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, config), 0)
    # detekce
    results = model.detect([image], verbose=0)

    r = results[0]
    colors = []
    for i in r['class_ids']:
        colors.append(c[i-1])
        
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                class_names, r['scores'], colors=colors)
    
    colors = []
    for i in gt_class_id:
        colors.append(c[i-1])
        
    visualize.display_instances(image, gt_bbox, gt_mask, gt_class_id, class_names, colors=colors)
    k = k + 1
    